In [1]:
from datetime import datetime

import numpy as np

import os

import pandas as pd

import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model

from time import time, gmtime, strftime

from bert.tokenization.bert_tokenization import FullTokenizer

from utils.models.modelUtils import loadMeanFromFile, calcAccConcatModel
from final_models import create_image_inceptionv3_model, create_model_meta, buildConcatModelMetaVisual

from utils.callbacks.MyCallbacks import MyCallbacks
from utils.datagenUtils.dual_modal.DataSeqMetaVisual import DataSequenceMetaImage

from utils.datagenUtils.datagenUtils import checkDataframe

from utils.telegramUtils.telegram_bot import telegram_send_message
from utils.textUtils.commentsProcessing import FakeDetectionDataCommentsTest, FakeDetectionDataCommentsTrainVal

from utils.callbacks.callbackUtils import plotTimesPerEpoch

from utils.fileDirUtils.fileDirUtils import createDirIfNotExists

from sklearn.metrics import accuracy_score


In [2]:
#Verbose settings:
verbose = False
TF_VERBOSE = 1 # 1 = Progress bar 2 = one line per epoch only!
TF_DETERMINISTIC_OPS = 1 # Makes everything also on GPU deterministic

# Classes:
NUM_CLASS = 2  # FAKE | NO FAKE

# Hyperparameters
GLOBAL_BATCH_SIZE = 128
EPOCHS = 10

# Optimizer parameters:
# Adam
LEARNING_RATE = 1e-5
BETA_1 = 0.9
BETA_2 = 0.999
EPSILON = 1e-8

#optimizers:

optimizer = Adam(LEARNING_RATE, BETA_1, BETA_2, EPSILON)


# Image Model  Parameters
IMG_WIDTH = 768
IMG_HEIGHT = 768
IMG_DEPTH = 3
IMG_SIZES = (IMG_WIDTH, IMG_HEIGHT)

# Retraining options
RETRAIN_WHOLE_MODEL = False
loadPretrained = False # If true model weights path must be set !

# Custom telegram send text 
CUSTOM_TEXT = f'Batch Size: {GLOBAL_BATCH_SIZE}, Epochs: {EPOCHS}, Optimizer: Adam, Learning Rate; {LEARNING_RATE}, Beta_1: {BETA_1}, Beta_2: {BETA_2}, Epsilon: {EPSILON}, Image Sizes: {IMG_SIZES}'


telegram_send_message(f'-----------------START-----------------')
print('START')
print(CUSTOM_TEXT)
telegram_send_message(CUSTOM_TEXT)

START
Batch Size: 128, Epochs: 10, Optimizer: Adam, Learning Rate; 1e-05, Beta_1: 0.9, Beta_2: 0.999, Epsilon: 1e-08, Image Sizes: (768, 768)


In [3]:
pathToRootModelDir = '/home/armin/repos/fkd-model-handling/models/best_models'

pathToImageModel = os.path.join(pathToRootModelDir, 'single_visual', 'weights-improvement-02-0.81.hdf5')
pathToMetaModel = os.path.join(pathToRootModelDir, 'single_meta', 'weights-improvement-100-0.62.hdf5')

pathToImagesTrain = '/home/armin/repos/FKD-Dataset/006_images_resized_2/train/' 
pathToCSVWithFileNamesAndLabelsTrain = '/home/armin/repos/FKD-Dataset/008_text_image_meta_label/train_text_image_meta_label.csv'

pathToImagesVal = '/home/armin/repos/FKD-Dataset/006_images_resized_2/val/' 
pathToCSVWithFileNamesAndLabelsVal = '/home/armin/repos/FKD-Dataset/008_text_image_meta_label/val_text_image_meta_label.csv'

pathToImagesTest = '/home/armin/repos/FKD-Dataset/006_images_resized_2/test/' 
pathToCSVWithFileNamesAndLabelsTest = '/home/armin/repos/FKD-Dataset/008_text_image_meta_label/test_text_image_meta_label.csv'

pathToMeans = '/home/armin/repos/FKD-Dataset/010_configs/means_resized_768.txt'

pathToModelWeights = ''

checkpointDir = '/home/armin/repos/FKD-Dataset/011_checkpoints'

root = '/home/armin/repos/fkd-model-handling/'
bert_model_dir = os.path.join(root, 'multi_cased_L-12_H-768_A-12')
bert_config_file = os.path.join(bert_model_dir, "bert_config.json")
bert_ckpt_file = os.path.join(bert_model_dir, "bert_model.ckpt") 

In [4]:
# Other ettings

# Time settings:
current_time = datetime.now().strftime("%Y-%m-%d_%H:%M")

#Checkpoint settings:
checkpoint_name = "model_concat_meta_image"

checkpointDir = os.path.join(checkpointDir, (checkpoint_name + '_' + current_time))

fileName="weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
filePath = os.path.join(checkpointDir, fileName)

In [5]:
# Callback Handling:
tensorboardDir = os.path.join(checkpointDir, 'tensorboard')

createDirIfNotExists(tensorboardDir)
createDirIfNotExists(checkpointDir)


callbacks_list = MyCallbacks(tensorboardDir, filePath, earlyStopping=True).createCheckpoints()

Directory /home/armin/repos/FKD-Dataset/011_checkpoints/model_concat_meta_image_2020-07-20_20:41/tensorboard does not exist, creating it instead!


In [6]:
df_train_ = pd.read_csv(pathToCSVWithFileNamesAndLabelsTrain, header=0, sep='\t')
df_test_ = pd.read_csv(pathToCSVWithFileNamesAndLabelsTest, header=0, sep='\t')
df_val_ = pd.read_csv(pathToCSVWithFileNamesAndLabelsVal, header=0, sep='\t')

df_train_['2_way_label'] = df_train_['2_way_label'].apply(lambda x: np.array(x))
df_test_['2_way_label'] = df_test_['2_way_label'].apply(lambda x: np.array(x))
df_val_['2_way_label'] = df_val_['2_way_label'].apply(lambda x: np.array(x))

df_train = df_train_
df_test = df_test_
df_val = df_val_

# df_train = df_train[:8000]
# df_test = df_test[:8000]
# df_val = df_val[:8000]

df_train = checkDataframe(df_train, GLOBAL_BATCH_SIZE)
df_val = checkDataframe(df_val, GLOBAL_BATCH_SIZE)
df_test = checkDataframe(df_test, GLOBAL_BATCH_SIZE)


In [7]:
# tokenizer = FullTokenizer(vocab_file=os.path.join(bert_model_dir, "vocab.txt"))


# all_text_data = FakeDetectionDataCommentsTrainVal(df_train, df_val, tokenizer, [0,1], MAX_SEQUENCE_LENGTH)


# all_text_test = FakeDetectionDataCommentsTest(df_test, tokenizer, [0,1], MAX_SEQUENCE_LENGTH)


# train_x = all_text_data.train_x
# val_x = all_text_data.val_x
# test_x = all_text_test.test_x

In [8]:
meansOfDataset = loadMeanFromFile(pathToMeans, verbose)

train_seq = DataSequenceMetaImage(df_train, pathToImagesTrain, GLOBAL_BATCH_SIZE, IMG_SIZES, meansOfDataset)

test_seq = DataSequenceMetaImage(df_test, pathToImagesTest, GLOBAL_BATCH_SIZE, IMG_SIZES, meansOfDataset)

val_seq = DataSequenceMetaImage(df_val, pathToImagesVal, GLOBAL_BATCH_SIZE, IMG_SIZES, meansOfDataset)

STEP_SIZE_TRAIN = len(df_train) // GLOBAL_BATCH_SIZE
STEP_SIZE_TEST = len(df_test) // GLOBAL_BATCH_SIZE
STEP_SIZE_VAL = len(df_val) // GLOBAL_BATCH_SIZE

In [9]:
# def buildConcatModelMetaVisual(model_meta, model_image, NUM_CLASS):
#     model_meta_output = model_meta.get_layer('final_output').output
#     model_image_output = model_image.get_layer('img_dense_768').output
    
#     # Build new models, without softmax
#     model_image = Model(model_image.inputs, model_image_output)    
#     model_meta = Model(model_meta.inputs, model_meta_output)
    
#     x = Dense(128, activation = 'relu')(model_image.output)
#     x = Dense (32, activation = 'relu')(x)
#     x = Dense (6, activation = 'relu')(x)
#     concatenate = Concatenate()([model_meta.output, x]) # Fusion
#     x = Dense(12, activation = 'relu')(concatenate)
#     x = Dropout(0.4)(x)
#     output = Dense(NUM_CLASS, activation='softmax', name='final_multimodal_output')(x)
#     model_concat = Model([model_image.input, model_meta.input], output)
#     return model_concat

In [10]:
mirrored_strategy = tf.distribute.MirroredStrategy()
with mirrored_strategy.scope():
    # Preparing:
    model_image, model_image_custom_layer = create_image_inceptionv3_model(NUM_CLASS=NUM_CLASS,isPreTrained=True,pathToInceptionV3ModelWeights=pathToImageModel, isTrainable=RETRAIN_WHOLE_MODEL)
    model_meta, model_meta_custom_layer = create_model_meta(NUM_CLASS, 2, True, pathToMetaModel, isTrainable=RETRAIN_WHOLE_MODEL)
    
    model_concat = buildConcatModelMetaVisual(model_meta, model_image, NUM_CLASS)

    if verbose:
        print('--------------IMAGE MODEL --------------')
        model_image.summary()
        print('--------------Meta MODEL --------------')
        model_meta.summary()
        print('--------------CONCAT MODEL --------------')
        model_concat.summary()
        for l in model_concat.layers:
            print(l.name, l.trainable)
            
    if loadPretrained:
        model_concat.load_weights(pathToModelWeights)
    
    
    model_concat.compile(loss=keras.losses.SparseCategoricalCrossentropy(),
          optimizer=optimizer,
          metrics=['accuracy'])
    
    history = model_concat.fit(train_seq,
                    steps_per_epoch=STEP_SIZE_TRAIN,epochs=EPOCHS, 
                    validation_data=val_seq,
                    validation_steps=STEP_SIZE_VAL, 
                    callbacks=callbacks_list,
                    use_multiprocessing=False,
                    workers=12,
                    verbose=TF_VERBOSE
                   )

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task

In [11]:
# # For later model loading
# modelFile = '/home/armin/repos/FKD-Dataset/011_checkpoints/model_concat_title_image_2020-07-15_13:16/weights-improvement-02-0.91.hdf5'

# model_concat.load_weights(modelFile)

In [12]:
%%capture

plot_model(model_concat, to_file=os.path.join(checkpointDir,'multiple_inputs_image_text_meta_pretrained.png'), show_shapes=True, dpi=600, expand_nested=False)
plot_model(model_concat, to_file=os.path.join(checkpointDir,'multiple_inputs_image_text_meta_pretrained_nested.png'), show_shapes=True, dpi=600, expand_nested=True)

In [13]:
plotTimesPerEpoch(callbacks_list)
calcAccConcatModel(model_concat, val_seq, GLOBAL_BATCH_SIZE, 'Val')
calcAccConcatModel(model_concat, test_seq, GLOBAL_BATCH_SIZE, 'Test')

460/460 [==============================] - 1301s 3s/step
Val Accuracy is 0.8122112771739131
460/460 [==============================] - 1350s 3s/step
Test Accuracy is 0.815625
It took so long to train on one epoch: [86, 86, 87, 86] minutes


In [14]:
telegram_send_message(f'-----------------DONE-----------------')
print('done')

done
